In [ ]:
"""
Created on Mon Jan 09 10:16 2023

This is a script to cut out the T and S in the 50 km in front of the ice front for SMITH data

@author: Clara Burgard
"""

- calculate the distance to the ice front for the small domain in front of the ice shelf
- take the ocean points at distance of ~50 km of the ice front 

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
#from tqdm import tqdm
import gsw
import matplotlib.pyplot as plt
import multimelt.useful_functions as uf
import multimelt.T_S_profile_functions as tspf

from scipy.spatial import cKDTree


import itertools

import distributed
import glob

In [ ]:
client = distributed.Client(n_workers=12, dashboard_address=':8795', local_directory='/tmp', memory_limit='6GB')

In [ ]:
client

In [ ]:
%matplotlib qt5

READ IN THE DATA

In [ ]:
nemo_run = 'bi646' # 'bf663','bi646' 

In [ ]:
inputpath_data='/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'
inputpath_profiles='/bettik/burgardc/DATA/NN_PARAM/interim/T_S_PROF/SMITH_'+nemo_run+'/'
inputpath_isf='/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/SMITH_'+nemo_run+'/'

# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

PREPARE MASK AROUND FRONT (TO RUN WITHOUT DASK!)

In [ ]:
def distance_isf_points_from_line_small_domain(isf_points_da,line_points_da):
    
    """
    Compute the distance between ice shelf points and a line.
    
    This function computes the distance between ice shelf points and a line. This line can be the grounding
    line or the ice shelf front.
    
    Parameters
    ----------
    whole_domain : xarray.DataArray
        ice-shelf mask - all ice shelves are represented by a number, all other points (ocean, land) set to nan
    isf_points_da : xarray.DataArray
        array containing only points from one ice shelf
    line_points_da : xarray.DataArray
        mask representing the grounding line or ice shelf front mask corresponding to the ice shelf selected in ``isf_points_da``
        
    Returns
    -------
    xr_dist_to_line : xarray.DataArray
        distance of the each ice shelf point to the given line of interest
    """
    
    # add a common dimension 'grid' along which to stack
    stacked_isf_points = isf_points_da.stack(grid=['y', 'x'])
    stacked_line = line_points_da.stack(grid=['y', 'x'])
    
    # remove nans
    filtered_isf_points = stacked_isf_points[stacked_isf_points>0]
    filtered_line = stacked_line[stacked_line>0]
    
    # write out the y,x pairs behind the dimension 'grid'
    grid_isf_points = filtered_isf_points.indexes['grid'].to_frame().values.astype(float)
    grid_line = filtered_line.indexes['grid'].to_frame().values.astype(float)
    
    # create tree to line and compute distance
    tree_line = cKDTree(grid_line)
    dist_yx_to_line, _ = tree_line.query(grid_isf_points)
        
    # add the coordinates of the previous variables
    xr_dist_to_line = filtered_isf_points.copy(data=dist_yx_to_line)
    # put 1D array back into the format of the grid and put away the 'grid' dimension
    xr_dist_to_line = xr_dist_to_line.unstack('grid')
    
    return xr_dist_to_line

In [ ]:
file_mask_orig = xr.open_dataset(inputpath_data+'mask_variables_of_interest_allyy_Ant_stereo.nc')#, chunks={'x': 600, 'y': 600})
file_mask_orig = file_mask_orig.assign_coords({'time': range(1970, 1970+len(file_mask_orig.time))})#.chunk({'time': 1})
file_mask = uf.cut_domain_stereo(file_mask_orig, map_lim, map_lim).isel(time=0).squeeze().drop('time')

# only points below 1500 m
offshore = file_mask['Bathymetry_isf'] > 1500 # .drop('lon').drop('lat')
# only points above 1500 m
contshelf = file_mask['Bathymetry_isf'] <= 1500 # .drop('lon').drop('lat')

In [ ]:
for timet in range(2001,1970 + 72):
    print(timet)
    T_S_ocean_file = xr.open_dataset(inputpath_profiles+'T_S_theta_ocean_corrected_'+str(timet)+'.nc').drop('time')
    
    file_isf_orig  = xr.open_dataset(inputpath_isf+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(timet)+'.nc')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf).chunk(chunks={'x': 50, 'y': 50}).squeeze().drop('time')
    if 'labels' in file_isf.coords.keys():
        file_isf = file_isf.drop('labels')
    
    lon = file_isf['longitude']
    lat = file_isf['latitude']
    
    ocean = np.isfinite(T_S_ocean_file['theta_ocean'].isel(depth=0)).drop('depth')
     
    # NB: 5.0 x 1.75 is the effective resolution at 70S for a model of 1 degree resolution in longitude (assuming 5 delta X and a Mercator grid)
    mask_50km = (ocean & contshelf).load()
    
    lon_box = np.array([10.0])
    lat_box = np.array([3.5])

    close_region_around_isf_mask = tspf.mask_boxes_around_IF_new(lon, lat, mask_50km, 
                                    file_isf['front_min_lon'], file_isf['front_max_lon'], 
                                    file_isf['front_min_lat'], file_isf['front_max_lat'],  
                                    lon_box, lat_box, 
                                    file_isf['isf_name'])
    
    dist_list = [ ]
    for kisf in tqdm(file_isf['Nisf']):

            if (file_isf['IF_mask']==kisf).sum() > 0:
                region_to_cut_out = close_region_around_isf_mask.sel(Nisf=kisf).squeeze()
                region_to_cut_out = region_to_cut_out.where(region_to_cut_out > 0, drop=True)
                IF_region = file_isf['IF_mask'].where(file_isf['IF_mask']==kisf, drop=True)

                dist_from_front = distance_isf_points_from_line_small_domain(region_to_cut_out,IF_region)
                dist_list.append(dist_from_front)

    dist_all = xr.concat(dist_list, dim='Nisf').reindex_like(file_isf)
    dist_all.to_dataset(name='dist_from_front').to_netcdf(inputpath_profiles+'dist_to_ice_front_only_contshelf_'+str(timet)+'.nc')
    
    #del dist_all

In [ ]:
for timet in range(1970,2051):
    print(timet)
    T_S_ocean_file = xr.open_dataset(inputpath_profiles+'T_S_theta_ocean_corrected_'+str(timet)+'.nc').drop('time')
    
    nisf_list = []
    file_isf_orig  = xr.open_dataset(inputpath_isf+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(timet)+'.nc')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf).chunk(chunks={'x': 50, 'y': 50}).squeeze().drop('time')
    if 'labels' in file_isf.coords.keys():
        file_isf = file_isf.drop('labels')
    if 75 in large_isf:
        nisf_list.append(75)
    if 54 in file_isf.Nisf:
        nisf_list.append(54)
    file_isf = file_isf.sel(Nisf=nisf_list)
    
    lon = file_isf['longitude']
    lat = file_isf['latitude']
    
    ocean = np.isfinite(T_S_ocean_file['theta_ocean'].isel(depth=0)).drop('depth')
     
    # NB: 5.0 x 1.75 is the effective resolution at 70S for a model of 1 degree resolution in longitude (assuming 5 delta X and a Mercator grid)
    mask_50km = (ocean & contshelf).load()
    
    lon_box = np.array([10.0])
    lat_box = np.array([3.5])

    close_region_around_isf_mask = tspf.mask_boxes_around_IF_new(lon, lat, mask_50km, 
                                    file_isf['front_min_lon'], file_isf['front_max_lon'], 
                                    file_isf['front_min_lat'], file_isf['front_max_lat'],  
                                    lon_box, lat_box, 
                                    file_isf['isf_name'])
    
    dist_list = [ ]
    for kisf in file_isf['Nisf']:

            if (file_isf['IF_mask']==kisf).sum() > 0:
                region_to_cut_out = close_region_around_isf_mask.sel(Nisf=kisf).squeeze().load()
                region_to_cut_out = region_to_cut_out.where(region_to_cut_out > 0, drop=True)
                IF_region = file_isf['IF_mask'].where(file_isf['IF_mask']==kisf, drop=True)

                dist_from_front = distance_isf_points_from_line_small_domain(region_to_cut_out,IF_region)
                dist_list.append(dist_from_front)

    dist_all = xr.concat(dist_list, dim='Nisf').reindex_like(file_isf)
    dist_all.to_dataset(name='dist_from_front').to_netcdf(inputpath_profiles+'dist_to_ice_front_only_contshelf_'+str(timet)+'_only_AlexIsland.nc')
    
    #del dist_all

In [ ]:
for timet in tqdm(range(1970,2051)):
    file_AI = xr.open_dataset(inputpath_profiles+'dist_to_ice_front_only_contshelf_'+str(timet)+'_only_AlexIsland.nc')
    file_orig = xr.open_dataset(inputpath_profiles+'dist_to_ice_front_only_contshelf_'+str(timet)+'.nc')
    
    merged_file = xr.concat([file_orig.drop_sel(Nisf=75),file_AI], dim='Nisf')
    
    merged_file.to_netcdf(inputpath_profiles+'dist_to_ice_front_only_contshelf_'+str(timet)+'_merged75.nc')

COMPUTING THE MEAN PROFILES

CONTINENTAL SHELF

(needs ~ 18 x 6 GB of memory)

In [ ]:
mask_domain_distkm = 50000

In [ ]:
for yy in tqdm(range(1970,1970 + 72)):
    dist_to_front_file_yy = xr.open_dataset(inputpath_profiles+'dist_to_ice_front_only_contshelf_'+str(yy)+'_merged75.nc').chunk({'Nisf': 5})
    T_S_ocean_file_yy = xr.open_dataset(inputpath_profiles+'T_S_theta_ocean_corrected_'+str(yy)+'.nc').chunk({'depth': 5})
    
    dist_to_front = dist_to_front_file_yy['dist_from_front']
    mask_km = dist_to_front <= mask_domain_distkm
    ds_sum = (T_S_ocean_file_yy * mask_km).sum(['x','y'])
    
    mask_depth = T_S_ocean_file_yy['salinity_ocean'].squeeze().drop('time') > 0
    mask_all = mask_km & mask_depth
    
    mask_sum = mask_all.sum(['x','y'])
    mask_sum = mask_sum.load()
    
    ds_mean = ds_sum/mask_sum
    ds_mean.to_netcdf(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_'+str(yy)+'.nc')